In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
Joined_scores2 = dataiku.Dataset("Joined_scores2")
Joined_scores2_df = Joined_scores2.get_dataframe()


# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

In [0]:
#This function decodes onehotencoded columns that start with colname (and eliminates those columns)
def DeOneHotEncodeColumn(df,colname):
    #These are the columns we think are onehotencoded
    cols_tryout=[col for col in df.columns if colname+"_"==col[:len(colname)+1]]
    #The cols from cols_tryout aren´t necessarily binary, so we´ll check
    cols=[]
    for col in cols_tryout:
        EsBin=True
        i=0
        #This other version I think could be faster when there are only ones and 0s faster (python faster on vector ops)
        #unicos=df[col].unique()
        #if len(unicos)<=2:
        #    for a in unicos:
        #        if a not in [0,1]:
        #            EsBin=False

        while EsBin and i<df.shape[0]:
            if df[col][i]!=0 and df[col][i]!=1:
                EsBin=False
            i+=1
        if EsBin:
            cols.append(col)
    #If there aren´t at least 2 binary columns... what are u doing lol
    if len(cols)<2:
        return df
    sol=np.array(["" for i in range(df.shape[0])],dtype="object")
    #Let´s get the output of our decoded column ()
    for col in cols:
        sol+=np.array([col[len(colname)+1:] if x else "" for x in df[col]],dtype="object")

    #Let´s drop the columns we used and put the decoded column in place :)
    df[colname]=sol
    df=df.drop(cols,axis=1)
    return df

In [0]:
#Finds all the positions where char is in word
def findall(char,word):
    pos=[word.find(char)]
    suma=pos[0]+1
    while pos[-1]!=-1:
        pos.append(word[suma:].find(char))
        suma+=pos[-1]+1#(hay que sumar 1 por el "_" que hay que borrar)
    if pos[0]==-1:
        return pos
    pos=pos[:-1]
    suma=pos[0]
    for i in range(1,len(pos)):
        suma+=pos[i]+1#Hay que sumar 1 por los arrays empiezan en 0)
        pos[i]=suma
    return pos

In [0]:
#This function detects all the possible columns that have been one hot encoded, it does so by checking for _
#This is because when creating a OH of "EVENT" we get columns such as EVENT_SPAIN, EVENT_FRANCE....
#Since the column we´re looking for is EVENT_TYPE, we´re gonna make sure to be able to get the longest string that ends in _
#And that is at the beginning of multiple column names
def Detect_OHcols(df):
    cols=df.columns
    future_colnames=[]
    for i in range(len(cols)):
        pos=findall("_",cols[i])
        names=[cols[i][:p] for p in pos]
        name_final=""
        for name in names:
            possible_columns=[]
            long=len(name)
            for j in range(i+1,len(cols)):
                if name==cols[j][:long]:
                    possible_columns.append(cols[j])
            if len(possible_columns)>=1:
                name_final=name
            if name_final not in future_colnames and name_final!="":
                future_colnames.append(name_final)

    return future_colnames

In [0]:
#We´re gonna detect the columns, sort their names so that EVENT_TYPE is done before EVENT...and done :)
def DeOneHotEncode(df,exclude=[]):
    cols=pd.Series(Detect_OHcols(df))
    cols=cols.sort_values(ascending=False)
    for col in cols:
        if col not in exclude:
            df=DeOneHotEncodeColumn(df,col)
    return df

In [0]:
Joined_scores_DeOH_df = DeOneHotEncode(Joined_scores2_df,exclude=["pred"]) # For this sample code, simply copy input to output

In [0]:
# Write recipe outputs
Joined_scores_DeOH = dataiku.Dataset("Joined_scores_DeOH")
Joined_scores_DeOH.write_with_schema(Joined_scores_DeOH_df)